In [ ]:
# ssh -f django@bcpp3.bhp.org.bw -L5003:localhost:3306 -N
# ssh -f django@bhp066.bhp.org.bw -L5002:localhost:3306 -N

In [1]:
# List of missing subject identifier
missing_arvs = []
file_name = '/Users/ckgathi/Downloads/BHP_FILE/check19.csv'
f = open(file_name, 'r')
lines = f.readlines()
lines.pop(0)
for line in lines:
    line = line.strip()
    missing_arvs.append(line)

missing_arvs = list(set(missing_arvs))
print(len(missing_arvs))

164


In [10]:
# List of id for missing avrs data for HivCareAdherence
from bcpp_subject.models import HivCareAdherence
from django_pandas.io import read_frame

hiv_care_adherence = HivCareAdherence.objects.using('bhp066').filter(subject_visit__subject_identifier__in=missing_arvs)
print(hiv_care_adherence.count())
df = read_frame(hiv_care_adherence)
hiv_care_adherence_ids = df['id'].tolist()
ids = []
for id in hiv_care_adherence_ids:
    ids.append(id.hex)
ids = list(set(ids))

313


In [41]:
import MySQLdb
import pandas as pd
db_name = 'bcpp015'
connection = MySQLdb.connect(host='127.0.0.1', port=5003,user='root', passwd='cc3721b', db=f'{db_name}')
data = []

for idd in ids:
    sql = f"SELECT CONCAT(bcpp_subject_arv.short_name, bcpp_subject_hivcareadherence_arvs.hivcareadherence_id) as avr_id FROM bcpp_subject_arv INNER JOIN bcpp_subject_hivcareadherence_arvs ON bcpp_subject_arv.id = bcpp_subject_hivcareadherence_arvs.arv_id WHERE bcpp_subject_hivcareadherence_arvs.hivcareadherence_id = \'{idd}\';"
    df_arvs = pd.read_sql(sql, con=connection)
    if not df_arvs.empty:
        arvs_ids_list = df_arvs['avr_id'].tolist()
        data.append(arvs_ids_list)

[['Zidovudine236fd99360d945e5a855dffba10f18a8',
  'Nevirapine236fd99360d945e5a855dffba10f18a8'],
 ['Lopinavir/ritonavir158bfd8990b54f0b9289c072a74ab53e']]

In [ ]:
# Update central server
from bcpp_subject.models import Arv
for d in data:
    for hca_id in d:
        short_name = hca_id[:-32]
        obj_uuid = hca_id[-32:]
        try:
            hca = HivCareAdherence.objects.using('bhp066').get(id=hca_id)
        except HivCareAdherence.DoesNotExist:
            print(f'this is a problem it needs to be moved to the server {hca_id}')
        else:
            arv = Arv.objects.get(short_name=short_name)
            a1.arvs.add(arv)
            hca.using('bhp066').save()